In [1]:
import pandas as pd
from tqdm import tqdm
df_train = pd.read_csv('train.csv')
print('hello')

hello


In [ ]:
print('hello')

In [ ]:
test = pd.read_csv('example_test_files/test.csv')
revealed_targets = pd.read_csv('example_test_files/revealed_targets.csv')
sample_submission = pd.read_csv('example_test_files/sample_submission.csv')

In [ ]:
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_
                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')
    return df[features]
df_ = generate_features(df)

In [4]:
import lightgbm as lgb 
import xgboost as xgb 
import catboost as cbt 
import numpy as np 
import joblib 
import os 

# os.system('mkdir models')
TRAINING = False
model_path ='models'

N_fold = 3

if TRAINING:
    X = df_.values
    Y = df_train['target'].values

    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]

    index = np.arange(len(X))

models = []

def train(model_dict, i, modelname='lgb'):
    if TRAINING:
        model = model_dict[modelname]
        model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
                    eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
                    # verbose=10, 
                    # early_stopping_rounds=100
                    )
        models.append(model)
        joblib.dump(model, f"./models/{modelname}_{i}.model")
    else:
        models.append(joblib.load(f"{model_path}/{modelname}_{i}.model"))
    return 

model_dict = {
    'lgb': lgb.LGBMRegressor(objective='regression_l1', n_estimators=50),
    'xgb': xgb.XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=50),
    'cbt': cbt.CatBoostRegressor(objective='MAE', iterations=3000),

}

for i in tqdm(range(N_fold)):
    train(model_dict, i, 'lgb')
    # train(model_dict, 'xgb')
    # train(model_dict, 'cbt')

100%|██████████| 3/3 [00:00<00:00, 85.73it/s]


In [5]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [6]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = generate_features(test)
    
    sample_prediction['target'] = np.mean([model.predict(feat) for model in models], 0)
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = generate_features(test)
    print(feat)